### CS 156a, Final, P13-18
Author: Liting Xiao

In [1]:
import numpy as np
from sklearn.svm import SVC

In [2]:
# create target func f(x) = mx + b
def create_data_set(N):
    X = np.random.rand(2, N) * 2 - 1
    Y = np.sign(X[1] - X[0] + 0.25*np.sin(np.pi*X[0]))
    return X.T, Y

In [3]:
def RBFkernel(gamma, train_x, train_y, test_x, test_y, C=1.):
    svc = SVC(kernel='rbf', C=C, gamma=gamma)
    svc.fit(train_x, train_y)
    E_in = sum(svc.predict(train_x)!=train_y) / float(len(train_y))
    E_out = sum(svc.predict(test_x)!=test_y) / float(len(test_y))
    return E_in, E_out

In [4]:
# regular RBF method
def rbfregular(mu, gamma, X, Y):
    matrix = np.zeros((len(X), len(mu)))
    for i in range(len(X)):
        for j in range(len(mu)):
            matrix[i][j] = np.exp(-gamma * np.linalg.norm(X[i] - mu[j])**2)
    pinv1 = np.linalg.pinv(np.matmul(matrix.T, matrix))
    return np.matmul(np.matmul(pinv1, matrix.T), Y)

def get_rbfreg_error(w, gamma, mu, x, y):
    error = 0
    for n in range(len(x)):
        pred = 0
        for k in range(len(w)):
            pred += w[k] * np.exp(-gamma*np.linalg.norm(x[n]-mu[k])**2)
        pred = np.sign(pred)
        if np.not_equal(pred, y[n]): error += 1
    return error / float(len(y))

In [5]:
# k-means clustering
def cluster_points(X, mu):
    clusters  = {}
    for x in X:
        bestmukey = min([(i[0], np.linalg.norm(x-mu[i[0]])) \
                    for i in enumerate(mu)], key=lambda t:t[1])[0]
        try:
            clusters[bestmukey].append(x)
        except KeyError:
            clusters[bestmukey] = [x]
    return clusters
 
def reevaluate_centers(mu, clusters):
    newmu = []
    keys = sorted(clusters.keys())
    for k in keys:
        newmu.append(np.mean(clusters[k], axis = 0))
    return newmu
 
def has_converged(mu, oldmu):
    return set([tuple(a) for a in mu]) == set([tuple(a) for a in oldmu])
 
def find_centers(X, K):
    # Initialize to K random centers
    oldmu = X[np.random.randint(X.shape[0], size=K)]
    mu = X[np.random.randint(X.shape[0], size=K)]
    while not has_converged(mu, oldmu):
        oldmu = mu
        # Assign all points in X to clusters
        clusters = cluster_points(X, mu)
        # Reevaluate centers
        mu = reevaluate_centers(oldmu, clusters)
    return mu

#### P13

In [6]:
count = 0
for i in range(1000):
    x, y = create_data_set(100)
    E_in, _ = RBFkernel(1.5, x, y, x, y, C=1e8)
    if E_in != 0: count += 1
print('The percentage that is not separable by RBF kernel: {}%'
      .format(count/10.))

The percentage that is not separable by RBF kernel: 0.0%


#### P14-15

In [7]:
for K in [9, 12]:
    count = 0
    for i in range(1000):
        train_x, train_y = create_data_set(100)
        test_x, test_y = create_data_set(100)
        mu_rbf = find_centers(train_x, K)
        w_rbf = rbfregular(mu_rbf, 1.5, train_x, train_y)
        E_out_rbf = get_rbfreg_error(w_rbf, 1.5, mu_rbf, test_x, test_y)
        _, E_out_ker = RBFkernel(1.5, train_x, train_y, test_x, test_y, C=1e8)
        if E_out_ker < E_out_rbf: count += 1
    print('The percentage that kernel beats the regular when K={}: {}%'
          .format(K, count/10.))

The percentage that kernel beats the regular when K=9: 75.8%
The percentage that kernel beats the regular when K=12: 63.3%


#### P16

In [8]:
in_count, out_count = 0, 0
for i in range(1000):
    E_in, E_out = {}, {}
    for K in [9, 12]:
        train_x, train_y = create_data_set(100)
        test_x, test_y = create_data_set(100)
        mu_rbf = find_centers(train_x, K)
        w_rbf = rbfregular(mu_rbf, 1.5, train_x, train_y)
        E_in[K] = get_rbfreg_error(w_rbf, 1.5, mu_rbf, train_x, train_y)
        E_out[K] = get_rbfreg_error(w_rbf, 1.5, mu_rbf, test_x, test_y)
    if E_in[9] > E_in[12]: in_count += 1
    if E_out[9] > E_out[12]: out_count += 1
print('The percentage that K=12 beats K=9:')
print('    E_in: {}%; E_out: {}%'.format(in_count/10., out_count/10.))

The percentage that K=12 beats K=9:
    E_in: 64.6%; E_out: 55.6%


#### P17

In [9]:
in_count, out_count = 0, 0
for i in range(1000):
    E_in, E_out = {}, {}
    for g in [1.5, 2]:
        train_x, train_y = create_data_set(100)
        test_x, test_y = create_data_set(100)
        mu_rbf = find_centers(train_x, 9)
        w_rbf = rbfregular(mu_rbf, g, train_x, train_y)
        E_in[g] = get_rbfreg_error(w_rbf, g, mu_rbf, train_x, train_y)
        E_out[g] = get_rbfreg_error(w_rbf, g, mu_rbf, test_x, test_y)
    if E_in[1.5] > E_in[2]: in_count += 1
    if E_out[1.5] > E_out[2]: out_count += 1
print('The percentage that gamma=2 beats gamma=1.5:')
print('    E_in: {}%; E_out: {}%'.format(in_count/10., out_count/10.))

The percentage that gamma=2 beats gamma=1.5:
    E_in: 36.3%; E_out: 42.4%


#### P18

In [10]:
count = 0
for i in range(1000):
    x, y = create_data_set(100)
    mu_rbf = find_centers(x, 9)
    w_rbf = rbfregular(mu_rbf, 1.5, x, y)
    E_in = get_rbfreg_error(w_rbf, 1.5, mu_rbf, x, y)
    if E_in == 0: count += 1
print('The percentage that regular RBF achieves E_in=0: {}%'
      .format(count/10.))

The percentage that regular RBF achieves E_in=0: 1.7%
